# Simulación de Montecarlo para estimar el riesgo de fallo en un talud
***

### **Editado por: Kevin Alexander Gómez**
#### Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)
***

### **Introducción**
Este Notebook es parte del [**Manual de Python aplicado a la Geología**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología.

### **¿Qué voy a hacer?**
Usando este manual, desarrollarás código en Python para realizar una simulación de Montecarlo aplicada a la estimación de estabilidad en taludes naturales (predicciones básicas de riesgo de deslizamiento).

### **¿Qué voy a aprender?**

- Aprenderás a desarrollar código usando Python.
- Aprenderás a usar funciones y `NamedTuples` (tuplas nombradas).
- Aprenderás a realizar una simulación de Montecarlo.

### **¿Qué voy a necesitar?**

- Este manual.
> **Nota**: Este documento es un archivo de formato `.ipynb` y solo puedes interactuar con este notebook siguiendo alguna de estas opciones:
> 
> - A través de un navegador web, usando la aplicación de `Google Colab`.
> - Usando `Binder`, una aplicación web que permite ejecutar código arbitrario dentro de un entorno virtual (similar a `Google Colab`).
> - A través de un editor de código instalado en tu computadora, como por ejemplo: `Jupyter Lab`, `Jupyter Notebook` o `Visual Studio Code`.
- Conocimientos básicos en geología, estadística y álgebra lineal.
- Perseverancia para aprender cada tema y creatividad para resolver problemas.

<br>

## **Índice**
***

1. [Predicción del riesgo de deslizamiento](#parte1)
2. [Modelo de estabilidad de talud infinito](#parte2)
3. [Simulación de Montecarlo para el análisis de incertidumbre](#parte3)
***

<a id="parte1"></a>

## **1. Predicción del riesgo de deslizamiento**
***
Uno de los riesgos geológicos más frecuentes en zonas montañosas o accidentadas es el <span style="color:gold">deslizamiento de taludes naturales</span>.

Estos deslizamientos pueden conducir al colapso de terraplenes arrastrando viviendas y vehículos, también pueden provocar la ruptura de diques y generar inundaciones.

Para taludes naturales con un suelo delgado y superficial, el factor de estabilidad del talud es (según [Skempton y DeLory, 1957](https://www.issmge.org/uploads/publications/1/41/1957_02_0074.pdf)):

$\Large F = \frac{c}{\gamma z \sin(\alpha) \cos(\alpha)} + \frac{\tan(\phi)}{\tan(\alpha)} - m \frac{\gamma_w \tan(\phi)}{\gamma \tan(\alpha)}$
 

Donde:
- $c$ es la cohesión del suelo
- $\phi$ es el ángulo de fricción interna del suelo
- $\gamma$ es el peso específico del suelo
- $\gamma_w$ es el peso específico del agua
- $\alpha$ es la inclinación del talud con respecto a la horizontal, mayor a 0
- $z$ es la profundidad de la superficie de deslizamiento
- $z_w$ es la altura del nivel freático por encima de la superficie de deslizamiento
- $m$ es el grado de saturación del suelo, donde\
$\Large m = \frac{z_w}{z}$\
  Cuando $m = 1$, la profundidad del nivel freático ha alcanzado la superficie.

Esta ecuación es conocida como el <span style="color:lightgreen">modelo de estabilidad de talud infinito</span>.\
En este modelo, se analiza un bloque superficial con un determinado espesor y una altura de nivel freático, y se supone una falla paralela a la superficie del terreno.

<img src="resources/infinite_slope_model.png" alt="Modelo de talud infinito" width="600"/>


El criterio de falla ocurre cuando <span style="color:lightgreen">$F < 1$</span>.\
Para <span style="color:lightgreen">$1 < F < 1.25$</span> el talud es moderadamente susceptible.\
Para <span style="color:lightgreen">$1.25 < F < 1.5$</span> el talud es ligeramente susceptible.\
Por último, para <span style="color:lightgreen">$F > 1.5$</span> el talud se considera no susceptible o estable.

<br>

### Nota
En una función, podemos señalar el tipo de dato que le corresponde a sus parámetros y también a su resultado:

In [ ]:
def saludar(nombre: str) -> str:
    return "Hola " + nombre

In [ ]:
saludar("Python")

Podemos usar objetos para referenciar colecciones de datos personalizadas. Para esto usaremos el módulo `typing` de Python.

In [ ]:
from typing import List

In [ ]:
Vector = List[float]

In [ ]:
def scale(scalar: float, vector: Vector) -> Vector:
    return [scalar * num for num in vector]

In [ ]:
scale(2.0, [1.0, -4.2, 5.4])

<a id="parte2"></a>

## **2. Modelo de estabilidad de talud infinito**
***

In [ ]:
import math
from typing import NamedTuple

import numpy as np
import matplotlib.pyplot as plt
plt.style.use("bmh")

import scipy.stats as stats

Ahora, crearemos una tupla nombrada (`NamedTuple`) que contenga unos parámetros iniciales del modelo de estabilidad:

In [ ]:
class Params(NamedTuple):
    z: float = 5
    z_w: float = 2.5
    m: float = 0.5
    c: float = 25
    phi: float = math.radians(30)
    alpha: float = math.radians(35)
    gamma: float = 20
    gamma_w: float = 9.81

In [ ]:
def slope_stability(p: Params):
    c1 = p.c / (p.gamma * p.z * math.sin(p.alpha) * math.cos(p.alpha))
    c2 = math.tan(p.phi) / math.tan(p.alpha)
    c3 = p.m * p.gamma_w * math.tan(p.phi) / (p.gamma * math.tan(p.alpha))
    return c1 + c2 - c3

Usando los parámetros iniciales, obtenemos un valor de <span style="color:lightgreen">$F = 1.15$</span>.\
Esto significa que el talud es moderadamente susceptible.

In [ ]:
slope_stability(Params())

Ahora, analizaremos el modelo de estabilidad de una manera intuitiva.\
Para esto, usaremos los parámetros de inclinación ($\alpha$) y grado de saturación del suelo ($m$).\
Vamos a verificar que el riesgo aumenta ($F$ disminuye) cuando alguno de estos dos parámetros aumenta su valor. Los demás valores iniciales permanecerán igual.

### **2.1. Ángulo de inclinación ($\alpha$)**
Crearemos una función que remplace el valor de inclinación dentro del conjunto de parámetros iniciales:

In [ ]:
def slope_stability_alpha(alpha: float) -> float:
    params = Params(alpha=math.radians(alpha))
    return slope_stability(params)

Estableceremos el rango de valores a evaluar y lo asignaremos a `support`, el resultado de la evaluación será asignado a `output`:

In [ ]:
support = np.linspace(1, 45, 100)
output = np.array([slope_stability_alpha(a) for a in support])

Graficamos los resultados:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(support, output, label="$F$")
ax.axhline(y=1, color="r", label=r"$F=1$")

ax.set_xlabel(r"Inclinación del talud ($\alpha$)", fontsize=16)
ax.set_ylabel(r"Factor de estabilidad del talud ($F$)", fontsize=16)

ax.legend(prop={"size": 25})

plt.show()

### **2.2. Grado de saturación del suelo ($m$)**
Crearemos una función que remplace el valor de inclinación dentro del conjunto de parámetros iniciales:

In [ ]:
def slope_stability_m(m: float) -> float:
    params = Params(m=m)
    return slope_stability(params)

Estableceremos el rango de valores a evaluar y lo asignaremos a `support`, el resultado de la evaluación será asignado a `output`:

In [ ]:
support = np.linspace(0.05, 0.95, 100)
outputs = np.array([slope_stability_m(m) for m in support])

Graficamos los resultados:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

ax.plot(support, outputs, label="$F$")
ax.axhline(y=1, color="r", label="$F=1$")

ax.set_xlabel(r"Grado de saturación del suelo ($m$)", fontsize=16)
ax.set_ylabel(r"Factor de estabilidad del talud ($F$)", fontsize=16)

ax.legend(prop={"size": 25})

plt.show()

<a id="parte3"></a>

## **3. Simulación de Montecarlo para el análisis de incertidumbre**

Los parámetros principales a analizar son la cohesión ($c$) y el ángulo de fricción interna ($\phi$) del suelo, con coeficientes de variación de 0.2 y 0.25 respectivamente.\
Asumiremos que estas variables tienen una distribución normal.\
Para ejecutar una simulación estocástica, definiremos una función que genere un conjunto aleatorio de valores iniciales para el cálculo de la estabilidad.

In [ ]:
def random_params(params: Params):
    return Params(c=stats.norm(25, 0.2*25).rvs(),
                  phi=stats.norm(math.radians(30), 0.25*math.radians(30)).rvs())

Al ejecutar la función `random_params`, obtendremos un conjunto aleatorio de valores iniciales:

In [ ]:
random_params(Params())

De igual manera, si evaluamos el conjunto aleatorio usando la función `slope_stability`:

In [ ]:
slope_stability(random_params(Params()))

Ahora, ejecutaremos una simulación de Montecarlo para estimar la distribución del factor de seguridad $F$:

In [ ]:
N = 10000
sim = np.zeros(N)

failed = 0
for i in range(N):
    sim[i] = slope_stability(random_params(Params()))
    if sim[i] < 1:
        failed += 1
        
print(f"Estimated failure probability: {failed/N:.3f}")

Recuerda que las simulaciones con un factor de establidad menor a 1 (a la izquierda de la línea roja en el histograma) representan deslizamientos de talud.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(sim, bins=40, density=True, alpha=0.5)
ax.axvline(x=1, color="r")
ax.set_xlabel("Factor de estabilidad del talud ($F$)")
plt.show()

Podemos estimar la probabilidad de fallo del talud en función del ángulo de inclinación del talud ($\alpha$):

In [ ]:
def failure_probability_alpha(alpha: float) -> float:
    N = 1000
    failed = 0
    for i in range(N):
        params = random_params(Params())
        if slope_stability(params._replace(alpha=math.radians(alpha))) < 1:
            failed += 1
    return failed / float(N)

Estableceremos el rango de valores a evaluar y lo asignaremos a `support`, el resultado de la evaluación será asignado a `output`:

In [ ]:
support = np.linspace(1, 55, 20)
output = np.array([failure_probability_alpha(a) for a in support])

Graficamos los resultados:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot([math.degrees(a) for a in support], output)
ax.set_xlabel(r"Ángulo de inclinación del talud ($\alpha$)")
ax.set_ylabel("Probabilidad de fallo")
plt.show()

De manera similar, podemos graficar la probabilidad de fallo del talud en función del grado de saturación del suelo ($m$):

In [ ]:
def failure_probability_m(m: float) -> float:
    N = 1000
    failed = 0
    for i in range(N):
        params = random_params(Params())
        if slope_stability(params._replace(m=m)) < 1:
            failed += 1
    return failed / float(N)

Estableceremos el rango de valores a evaluar y lo asignaremos a `support`, el resultado de la evaluación será asignado a `output`:

In [ ]:
support = np.linspace(0.05, 0.95, 20)
outputs = np.array([failure_probability_m(m) for m in support])

Graficamos los resultados:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
ax.plot(support, output)
ax.set_xlabel("Grado de saturación del suelo ($m$)")
ax.set_ylabel("Probabilidad de fallo")
plt.show()

***